In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten,\
BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import os
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
from skimage.transform import resize
from os import listdir, walk
from os.path import isfile, join
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K

import splitfolders

In [45]:
#!pip install split-folders

In [2]:
directory = "data/mushrooms/first_batch/"

In [3]:
subfolders_path = [ f.path for f in os.scandir("data/mushrooms/first_batch/") if f.is_dir() ]
subfolders_path

['data/mushrooms/first_batch/training_data',
 'data/mushrooms/first_batch/Auricularia_auricula-judae',
 'data/mushrooms/first_batch/test_data',
 'data/mushrooms/first_batch/Cuphophyllus_virgineus',
 'data/mushrooms/first_batch/Bjerkandera_adusta',
 'data/mushrooms/first_batch/valid_set',
 'data/mushrooms/first_batch/Coprinellus_micaceus',
 'data/mushrooms/first_batch/Clitocybe_nebularis']

In [5]:
subfolders = [ f.name for f in os.scandir("data/mushrooms/first_batch/") if f.is_dir() ]
subfolders

['training_data',
 'Auricularia_auricula-judae',
 'test_data',
 'Cuphophyllus_virgineus',
 'Bjerkandera_adusta',
 'valid_set',
 'Coprinellus_micaceus',
 'Clitocybe_nebularis']

In [6]:
# give the directory that includes folders of images and return three files split in train test validation
dir = 'data/mushrooms/second_batch/'
splitfolders.ratio(dir , output=dir + "output", seed=1337, ratio=(.75, 0.15,0.1)) 

Copying files: 5598 files [00:19, 293.69 files/s]


In [49]:
output

NameError: name 'output' is not defined

In [4]:
CLASSES = dict(zip(subfolders, np.arange(0, len(subfolders))))

NameError: name 'subfolders' is not defined

In [23]:
subdirectory = subfolders[0]
onlyfiles = [f for f in listdir(subdirectory) if isfile(join(subdirectory, f))]
#onlyfiles.remove('.DS_Store')

In [29]:
def get_label(key ,class_mush ):
    """
    You can get the labeled name of the mushroom by giving 
    the index or the index by giving the key
    """
    if type(key) == str:
        return class_mush[key]
    elif type(key) == int:
        return list(class_mush.keys())[list(class_mush.values()).index(key)]

In [ ]:
def fetch_files_paths(directory):
    subfolders_path = [ f.path for f in os.scandir(directory) if f.is_dir() ]
    CLASSES = dict(zip(subfolders_path, np.arange(0, len(subfolders_path))))
    return subfolders_path, CLASSES

In [31]:
def fetch_image(subdirectory):
    onlyfiles = [f for f in listdir(subdirectory) if isfile(join(subdirectory, f))]
    label = np.full((1, len(onlyfiles)), get_label(onlyfiles))#.ravel()
    return onlyfiles, label

    

In [ ]:
def read_images()

In [ ]:
def gray_reshape(directory, new_size = (28,28), gray =False):
    """
    give the list of photos with the correct pixel size you want to 
    downsize. If you want Gray then write True.
    """
    items_resized  =[]
    pictures = []
    labels = []
    mapping = {'plants' : 0,'coins' : 1,'faces' : 2,'cups' : 3,'glasses' : 4,'pens' : 5,'gestures' : 6,'cutlery' : 7,'plates' : 8, 
                'nail_polishes' : 9  ,'shoes' : 10 
                }
    onlyfolders = [f for f in listdir(directory)]
    for idx, folder in enumerate(onlyfolders[1:]):
        pictures = [f for f in listdir(directory + folder) if isfile(join(directory + folder , f))]
        for idx, picture in enumerate(pictures):
            item = cv2.imread(directory + folder +'/'+ picture )
            labels.append(mapping[folder])
            
            if gray == True:
                item_gray = cv2.cvtColor(item, cv2.COLOR_BGR2GRAY)
                item_resized_gray = resize(item_gray, new_size)
                items_resized.append(item_resized_gray)
            else:
                item_resized_colored = resize(item, new_size)
                items_resized.append(item_resized_colored)
    return items_resized, labels

In [15]:
train_path = 'data/mushrooms/second_batch/output/train/'
test_path = 'data/mushrooms/second_batch/output/test/'
valid_path = 'data/mushrooms/second_batch/output/val/'

In [26]:
def fetch_files_paths(directory):
    subfolders_path = [ f.path for f in os.scandir(directory) if f.is_dir() ]
    return subfolders_path

In [27]:
fetch_files_paths('data/mushrooms/second_batch/output')

['data/mushrooms/second_batch/output/test',
 'data/mushrooms/second_batch/output/train',
 'data/mushrooms/second_batch/output/val']

In [ ]:
train_path = 'data/mushrooms/second_batch/output/train/'
test_path = 'data/mushrooms/second_batch/output/test/'
valid_path = 'data/mushrooms/second_batch/output/val/'

In [17]:
train_datagen = ImageDataGenerator(rescale=1.0/255.0,validation_split=0.2) 
CLASSES = ['Daedaleopsis_confragosa', 'Fomes_fomentarius','Fomitopsis_pinicola','Ganoderma_applanatum','Gymnopilus_penetrans' ]
train_batches = train_datagen.flow_from_directory(directory=train_path, \
target_size=(224,224), classes=CLASSES, batch_size=64,subset='training')

validation_batches = train_datagen.flow_from_directory(directory=train_path, \
target_size=(224,224), classes=CLASSES, batch_size=64,subset='validation')

test_batches = ImageDataGenerator(rescale=1.0/255.0).flow_from_directory(\
directory=test_path, target_size=(224,224), classes=CLASSES, \
batch_size=64, shuffle=False)

Found 3359 images belonging to 5 classes.
Found 837 images belonging to 5 classes.
Found 565 images belonging to 5 classes.


In [21]:
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, decode_predictions, preprocess_input
from tensorflow.keras.preprocessing import image # Keras own inbuild image class
import os
from tensorflow.keras.models import Model
from tensorflow import keras

def model_ResNet50V2( learning_rate=0.001, cl = CLASSES):
    """
    This function returns a ResNet50V2 model with the last
    layer removed.
    """
    K.clear_session() # Always clear the session!

    base_model = ResNet50V2(
        weights='imagenet', 
        pooling='avg',      # applies global average pooling to the output of the last conv layer (like a flattening)
        include_top=False,   # !!!!! we only want to have the base, not the final dense layers 
        input_shape=(224, 224, 3)  
    )
    base_model.trainable = False # To freeze the model
    # Start building on top of the model
    model = keras.Sequential() # defining a new model
    model.add(base_model) # adding in the pretrained model without the fully connected layer
    model.add(keras.layers.Dense(64, activation='relu')) # adding in additional layers
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(128, activation='relu')) # adding in additional layers
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(5, activation='softmax')) #!!! Final layer with a length of 2, and softmax activation 
    # have a look at the trainable and non-trainable params statistic
    model.summary()
    model.compile(optimizer=keras.optimizers.Adam(learning_rate),
              loss=keras.losses.categorical_crossentropy,
              metrics=[keras.metrics.categorical_accuracy])

    return model

def fit_ResNet50V2(train_batches,validation_data ,epochs = 200, sav = True):
    """
    This function fits the model on the training data.
    """
    # observe the validation loss and stop when it does not improve after 3 iterations
    callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

    history = model.fit( x=train_batches,
            epochs=epochs, 
            verbose=1, 
            callbacks=[callback],
            validation_data = validation_data, )
    
    # save model
    if sav == True:
        model.save('models/ResNet50V2_test.h5')
        
    return history


In [22]:
model = model_ResNet50V2(cl =CLASSES)
history = fit_ResNet50V2(train_batches,validation_batches, epochs = 200)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 2048)              23564800  
                                                                 
 dense (Dense)               (None, 64)                131136    
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 128)               8320      
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 5)                 645       
                                                                 
Total params: 23,704,901
Trainable params: 140,101
Non-t